In [3]:
from datasets import load_dataset

gtzan = load_dataset("marsyas/gtzan", "all", trust_remote_code=True)
print(gtzan)

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 999
    })
})


In [4]:
# Create test/train split
gtzan_with_train_test_split = gtzan["train"].train_test_split(seed=42, shuffle=True, test_size=0.1)
print(gtzan_with_train_test_split)

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})


In [5]:
from transformers import AutoFeatureExtractor
from datasets import Audio

model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

model_sampling_rate = feature_extractor.sampling_rate

# cast audio to correct sampling rate
gtzan_with_correct_sampling_rate = gtzan_with_train_test_split.cast_column("audio", Audio(sampling_rate=model_sampling_rate))

In [6]:
import numpy as np

# Test feature extractor on first sample
sample = gtzan_with_correct_sampling_rate["train"][0]["audio"]

print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])

print(f"inputs keys: {list(inputs.keys())}")

print(
    f"Mean: {np.mean(inputs['input_values']):.3}, Variance: {np.var(inputs['input_values']):.3}"
)

Mean: 0.000185, Variance: 0.0493
inputs keys: ['input_values', 'attention_mask']
Mean: -7.45e-09, Variance: 1.0


In [7]:
max_duration = 30.0


def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

In [8]:
gtzan_encoded = gtzan_with_correct_sampling_rate.map(
    preprocess_function,
    remove_columns=["audio", "file"],
    batched=True,
    batch_size=1000,
    num_proc=1,
)
gtzan_encoded

DatasetDict({
    train: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 899
    })
    test: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 100
    })
})

In [9]:
# Rename "genre" to "label" for Trainer
gtzan_with_label_col = gtzan_encoded.rename_column("genre", "label")

# Create id2label and label2id for use later
id2label_fn = gtzan_encoded["train"].features["genre"].int2str

id2label = {
    str(i): id2label_fn(i)
    for i in range(len(gtzan_with_label_col["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}



In [10]:
from transformers import AutoModelForAudioClassification

num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from transformers import TrainingArguments

model_name = model_id.split("/")[-1]
batch_size = 8
gradient_accumulation_steps = 1
num_train_epochs = 10
output_dir=f"{model_name}-finetuned-gtzan"

training_args = TrainingArguments(
    output_dir=output_dir,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    report_to="none"
)

In [13]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)


In [12]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_with_label_col["train"],
    eval_dataset=gtzan_with_label_col["test"],
    processing_class=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

NameError: name 'compute_metrics' is not defined

In [ ]:
from transformers import pipeline

classifier = pipeline(
    "audio-classification",
    model=f"{output_dir}/checkpoint-1017",
    feature_extractor=feature_extractor,
    device=0,
)

output = classifier(
    [
        "audio_files/80s-retro-synth-wave-medium-2-version-223101.mp3",
        "audio_files/dreaming-of-reggae-217732.mp3",
        "audio_files/powerful-epic-rock-248903.mp3"
    ]
)

print(output)

[[{'score': 0.8359705805778503, 'label': 'disco'}, {'score': 0.07591427117586136, 'label': 'reggae'}, {'score': 0.047974858433008194, 'label': 'hiphop'}, {'score': 0.017098549753427505, 'label': 'pop'}, {'score': 0.008336302824318409, 'label': 'rock'}], [{'score': 0.8691219091415405, 'label': 'reggae'}, {'score': 0.07571885734796524, 'label': 'hiphop'}, {'score': 0.03892843797802925, 'label': 'blues'}, {'score': 0.006575872655957937, 'label': 'classical'}, {'score': 0.0038647756446152925, 'label': 'disco'}], [{'score': 0.5603878498077393, 'label': 'hiphop'}, {'score': 0.39278435707092285, 'label': 'metal'}, {'score': 0.011860944330692291, 'label': 'jazz'}, {'score': 0.007799150422215462, 'label': 'rock'}, {'score': 0.00732860853895545, 'label': 'disco'}]]
